In [1]:

#Generates a set of coefficients as a sage "vector"
def coefs_1d(N,N0,lab) :
    return vector([ var(lab+'%s'%i) for i in range(N0,N0+N) ])


#Generates a 1-D polynomial as a sage expreession based on a set of coefficients
def poly_1d(N,coefs,x) :
    return sum( vector([ coefs[i]*x^i for i in range(N) ]) )

latex.matrix_delimiters("[", "]")
latex.matrix_column_alignment(align='c')


In [3]:
var('rho_km3o2,rho_km1,rho_km1o2,rho_k,rho_kp1o2,rho_kp1,rho_kp3o2,dt,dx')
var('press_km2,press_km1,press_k,press_kp1,press_kp2')

# Assign gll points & weights over the domain [-0.5,0.5]
gllWts = vector([1/6 , 4/6 , 1/6])

# Create Pressure values over the 5-cell stencil
press = vector([press_km2,press_km1,press_k,press_kp1,press_kp2])

# Stencil to coefs left
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([0*x for i in range(3)])
constr[0] = integrate(p,x,-5*dx/2,-3*dx/2)/dx
constr[1] = integrate(p,x,-3*dx/2,-1*dx/2)/dx
constr[2] = integrate(p,x,-1*dx/2, 1*dx/2)/dx
coefs_to_constr = jacobian(constr,coefs)
s2c_left = coefs_to_constr^-1

# Stencil to coefs middle
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([0*x for i in range(3)])
constr[0] = integrate(p,x,-3*dx/2,-1*dx/2)/dx
constr[1] = integrate(p,x,-1*dx/2, 1*dx/2)/dx
constr[2] = integrate(p,x, 1*dx/2, 3*dx/2)/dx
coefs_to_constr = jacobian(constr,coefs)
s2c_middle = coefs_to_constr^-1

# Stencil to coefs right
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([0*x for i in range(3)])
constr[0] = integrate(p,x,-1*dx/2, 1*dx/2)/dx
constr[1] = integrate(p,x, 1*dx/2, 3*dx/2)/dx
constr[2] = integrate(p,x, 3*dx/2, 5*dx/2)/dx
coefs_to_constr = jacobian(constr,coefs)
s2c_right = coefs_to_constr^-1

##########################
# p_star in left cell
##########################
# Create matrix to convert 
coefs = s2c_left * press[0:3]
p = poly_1d(3,coefs,x)
pnt1 = diff(p,x).subs(x=-3*dx/2)
pnt2 = diff(p,x).subs(x=-1*dx)
pnt3 = diff(p,x).subs(x=-1*dx/2)
pstar_left = gllWts[0]*rho_km3o2*pnt1 + gllWts[1]*rho_km1*pnt2 + gllWts[2]*rho_km1o2*pnt3

show(pstar_left.simplify_full())

##########################
# p_star in middle cell
##########################
# Create matrix to convert 
coefs = s2c_middle * press[1:4]
p = poly_1d(3,coefs,x)
pnt1 = diff(p,x).subs(x=-1*dx/2)
pnt2 = diff(p,x).subs(x= 0*dx)
pnt3 = diff(p,x).subs(x= 1*dx/2)
pstar_middle = gllWts[0]*rho_km1o2*pnt1 + gllWts[1]*rho_k*pnt2 + gllWts[2]*rho_kp1o2*pnt3

##########################
# p_star in right cell
##########################
# Create matrix to convert 
coefs = s2c_right * press[2:5]
p = poly_1d(3,coefs,x)
pnt1 = diff(p,x).subs(x=1*dx/2)
pnt2 = diff(p,x).subs(x=1*dx)
pnt3 = diff(p,x).subs(x=3*dx/2)
pstar_right = gllWts[0]*rho_kp1o2*pnt1 + gllWts[1]*rho_kp1*pnt2 + gllWts[2]*rho_kp3o2*pnt3


##################################
# Integrated derivative of pstar
##################################
pstar = vector([pstar_left,pstar_middle,pstar_right])
coefs = s2c_middle * pstar
p = poly_1d(3,coefs,x)
expr = dt * ( p.subs(x=dx/2) - p.subs(x=-dx/2) )

# Get the weights to the pressure terms
weights = jacobian(expr,press)
print("Vertical pressure weights for the Poisson problem:")
show(weights.simplify_full().transpose())

print("\nHorizontal pressure weights for the Poisson problem:")
show(weights.subs(rho_km3o2=rho_k,rho_km1=rho_k,rho_km1o2=rho_k,rho_kp1o2=rho_k,rho_kp1=rho_k,rho_kp3o2=rho_k).transpose())

# Confirm the weights are symmetric in the end
if (weights*vector([1,1,1,1,1])).simplify_full() != 0:
    print("ERROR: pressure weights are not consistent")

########################################
# Now compute momentum divergence term
########################################
var('mom_km1,mom_k,mom_kp1')
mom = vector([mom_km1,mom_k,mom_kp1])
coefs = s2c_middle * mom
p = poly_1d(3,coefs,x)
expr = p.subs(x=dx/2) - p.subs(x=-dx/2)
weights = jacobian(expr,mom)
print("\nMomentum weights:")
show(weights.simplify_full().transpose())

if (weights*vector([1,1,1])).simplify_full() != 0 :
    print("ERROR: momentum weights are not consistent")
    
########################################
# Pressure application weights
########################################
print("Vertical pressure weights for pressure gradient application:")
expr = pstar_middle
weights = jacobian(expr,press)
show(weights.simplify_full().transpose())

print("Horizontal pressure weights for pressure gradient application:")
expr = pstar_middle
weights = jacobian(expr,press)
show(weights.subs(rho_km3o2=rho_k,rho_km1=rho_k,rho_km1o2=rho_k,rho_kp1o2=rho_k,rho_kp1=rho_k,rho_kp3o2=rho_k).simplify_full().transpose())

1/6*(2*(press_k - press_km2)*rho_km1 + (press_k - press_km1)*rho_km1o2 + (press_km1 - press_km2)*rho_km3o2)/dx

Vertical pressure weights for the Poisson problem:


[                               1/12*(2*dt*rho_km1 + dt*rho_km3o2)/dx]
[                               1/12*(dt*rho_km1o2 - dt*rho_km3o2)/dx]
[-1/12*(2*dt*rho_km1 + dt*rho_km1o2 + 2*dt*rho_kp1 + dt*rho_kp1o2)/dx]
[                               1/12*(dt*rho_kp1o2 - dt*rho_kp3o2)/dx]
[                               1/12*(2*dt*rho_kp1 + dt*rho_kp3o2)/dx]


Horizontal pressure weights for the Poisson problem:


[ 1/4*dt*rho_k/dx]
[               0]
[-1/2*dt*rho_k/dx]
[               0]
[ 1/4*dt*rho_k/dx]

...

[-1/2]
[   0]
[ 1/2]

Vertical pressure weights for pressure gradient application:


[                             0]
[ -1/6*(2*rho_k + rho_km1o2)/dx]
[1/6*(rho_km1o2 - rho_kp1o2)/dx]
[  1/6*(2*rho_k + rho_kp1o2)/dx]
[                             0]

Horizontal pressure weights for pressure gradient application:


[            0]
[-1/2*rho_k/dx]
[            0]
[ 1/2*rho_k/dx]
[            0]

In [ ]:
var('rho_km3o2,rho_km1,rho_km1o2,rho_k,rho_kp1o2,rho_kp1,rho_kp3o2,dt,dx')
var('press_km2,press_km1,press_k,press_kp1,press_kp2')

# Assign gll points & weights over the domain [-0.5,0.5]
gllWts = vector([1/6 , 4/6 , 1/6])

# Create Pressure values over the 5-cell stencil
press = vector([press_km2,press_km1,press_k,press_kp1,press_kp2])

# Stencil to coefs left
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([0*x for i in range(3)])
constr[0] = integrate(p,x,-5*dx/2,-3*dx/2)/dx
constr[1] = integrate(p,x,-3*dx/2,-1*dx/2)/dx
constr[2] = integrate(p,x,-1*dx/2, 1*dx/2)/dx
coefs_to_constr = jacobian(constr,coefs)
s2c_left = coefs_to_constr^-1

# Stencil to coefs middle
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([0*x for i in range(3)])
constr[0] = integrate(p,x,-3*dx/2,-1*dx/2)/dx
constr[1] = integrate(p,x,-1*dx/2, 1*dx/2)/dx
constr[2] = integrate(p,x, 1*dx/2, 3*dx/2)/dx
coefs_to_constr = jacobian(constr,coefs)
s2c_middle = coefs_to_constr^-1

# Stencil to coefs right
coefs = coefs_1d(3,0,'a')
p = poly_1d(3,coefs,x)
constr = vector([0*x for i in range(3)])
constr[0] = integrate(p,x,-1*dx/2, 1*dx/2)/dx
constr[1] = integrate(p,x, 1*dx/2, 3*dx/2)/dx
constr[2] = integrate(p,x, 3*dx/2, 5*dx/2)/dx
coefs_to_constr = jacobian(constr,coefs)
s2c_right = coefs_to_constr^-1

##########################
# p_star in left cell
##########################
# Create matrix to convert 
coefs = s2c_left * press[0:3]
p = poly_1d(3,coefs,x)
pnt1 = diff(p,x).subs(x=-3*dx/2)
pnt2 = diff(p,x).subs(x=-1*dx)
pnt3 = diff(p,x).subs(x=-1*dx/2)
pstar_left = gllWts[0]*rho_km3o2*pnt1 + gllWts[1]*rho_km1*pnt2 + gllWts[2]*rho_km1o2*pnt3

show(pstar_left.simplify_full())

##########################
# p_star in middle cell
##########################
# Create matrix to convert 
coefs = s2c_middle * press[1:4]
p = poly_1d(3,coefs,x)
pnt1 = diff(p,x).subs(x=-1*dx/2)
pnt2 = diff(p,x).subs(x= 0*dx)
pnt3 = diff(p,x).subs(x= 1*dx/2)
pstar_middle = gllWts[0]*rho_km1o2*pnt1 + gllWts[1]*rho_k*pnt2 + gllWts[2]*rho_kp1o2*pnt3

##########################
# p_star in right cell
##########################
# Create matrix to convert 
coefs = s2c_right * press[2:5]
p = poly_1d(3,coefs,x)
pnt1 = diff(p,x).subs(x=1*dx/2)
pnt2 = diff(p,x).subs(x=1*dx)
pnt3 = diff(p,x).subs(x=3*dx/2)
pstar_right = gllWts[0]*rho_kp1o2*pnt1 + gllWts[1]*rho_kp1*pnt2 + gllWts[2]*rho_kp3o2*pnt3


##################################
# Integrated derivative of pstar
##################################
pstar = vector([pstar_left,pstar_middle,pstar_right])
coefs = s2c_middle * pstar
p = poly_1d(3,coefs,x)
expr = dt * ( p.subs(x=dx/2) - p.subs(x=-dx/2) )

# Get the weights to the pressure terms
weights = jacobian(expr,press)
print("Vertical pressure weights for the Poisson problem:")
show(weights.simplify_full().transpose())

print("\nHorizontal pressure weights for the Poisson problem:")
show(weights.subs(rho_km3o2=rho_k,rho_km1=rho_k,rho_km1o2=rho_k,rho_kp1o2=rho_k,rho_kp1=rho_k,rho_kp3o2=rho_k).transpose())

# Confirm the weights are symmetric in the end
if (weights*vector([1,1,1,1,1])).simplify_full() != 0:
    print("ERROR: pressure weights are not consistent")

########################################
# Now compute momentum divergence term
########################################
var('mom_km1,mom_k,mom_kp1')
mom = vector([mom_km1,mom_k,mom_kp1])
coefs = s2c_middle * mom
p = poly_1d(3,coefs,x)
expr = p.subs(x=dx/2) - p.subs(x=-dx/2)
weights = jacobian(expr,mom)
print("\nMomentum weights:")
show(weights.simplify_full().transpose())

if (weights*vector([1,1,1])).simplify_full() != 0 :
    print("ERROR: momentum weights are not consistent")
    
########################################
# Pressure application weights
########################################
print("Vertical pressure weights for pressure gradient application:")
expr = pstar_middle
weights = jacobian(expr,press)
show(weights.simplify_full().transpose())

print("Horizontal pressure weights for pressure gradient application:")
expr = pstar_middle
weights = jacobian(expr,press)
show(weights.subs(rho_km3o2=rho_k,rho_km1=rho_k,rho_km1o2=rho_k,rho_kp1o2=rho_k,rho_kp1=rho_k,rho_kp3o2=rho_k).simplify_full().transpose())